# Ingesting STAC metadata in APEx Product Catalogue

This notebook demonstrates how to add STAC metadata to an [APEx Product Catalogue](../instantiation/catalog.qmd).
Since STAC is a widely adopted specification, existing STAC documentation may also be useful for understanding its broader applications.

## Creating STAC metadata from scratch

In this first part, we provide a basic example of how to manually create STAC metadata from scratch.
However, in most real-world cases, this approach is not needed nor recommended. Instead, various tools and platforms will generate STAC metadata for you.

The STAC metadata in this example is entirely fictional and not intended for practical use.
It does not demonstrate how to create high-quality, FAIR-compliant metadata but serves purely as a demonstration of the STAC API and its ease of use.
Most of the more complex examples and tools will avoid this type of direct interaction.

In [1]:
import requests
from rio_stac import create_stac_item
from owslib.ogcapi.records import Records
from owslib.util import Authentication
from getpass import getpass
import json

For ease of use, we define the URL of the APEx catalogue here. This allows us to reuse it as a reference throughout the notebook.

In [2]:
CATALOGUE = "https://catalogue.demo.apex.esa.int" 

### Authentication for APEx Product Catalogue

To interact with an APEx Product Catalogue, you need a valid authentication token. This token ensures that you can write metadata to an APEx-initialized STAC catalogue, provided you have been granted sufficient access rights.

To obtain an authentication token, follow the dedicated guide on [generating your OIDC token](token.md). This step is essential for securely accessing and modifying catalogue records.

In [3]:
# Helper class to support the authentication with owslib and requests libraries
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r
    

token = getpass("Please provide your OIDC token here")
auth = BearerAuth(token)

In [4]:
r = Records(CATALOGUE,auth=Authentication(auth_delegate=auth))
print(json.dumps(r.collections(), indent=2))

{
  "collections": [],
  "links": [
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://catalogue.demo.apex.esa.int/"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://catalogue.demo.apex.esa.int/"
    },
    {
      "rel": "self",
      "type": "application/json",
      "href": "https://catalogue.demo.apex.esa.int/collections"
    }
  ]
}


### Exploring the dataset
Before creating the STAC metadata, it's useful to explore the dataset. This helps in understanding key properties such as spatial extent, resolution, and format.

A quick way to retrieve metadata from a representative asset is by using the `gdalinfo` command. Running this command provides useful details, including:

- Projection and coordinate system
- Bounding box and spatial resolution
- Number of bands and data type
- ...
  
This information will guide the creation of accurate STAC metadata for the dataset.

In [5]:
!GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR gdalinfo -json "/vsicurl/https://eoresults.esa.int/d/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012/2020/01/01/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012-SK_2020/SK_L1_pp_2020.tif"

{
  "description":"/vsicurl/https://eoresults.esa.int/d/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012/2020/01/01/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012-SK_2020/SK_L1_pp_2020.tif",
  "driverShortName":"GTiff",
  "driverLongName":"GeoTIFF",
  "files":[
    "/vsicurl/https://eoresults.esa.int/d/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012/2020/01/01/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012-SK_2020/SK_L1_pp_2020.tif"
  ],
  "size":[
    56822,
    28119
  ],
  "coordinateSystem":{
    "wkt":"PROJCRS[\"ETRS89-extended / LAEA Europe\",\n    BASEGEOGCRS[\"ETRS89\",\n        ENSEMBLE[\"European Terrestrial Reference System 1989 ensemble\",\n            MEMBER[\"European Terrestrial Reference Frame 1989\"],\n            MEMBER[\"European Terrestrial Reference Frame 1990\"],\n            MEMBER[\"European Terrestrial Reference Frame 1991\"],\n            MEMBER[\"European Terrestrial Reference Frame 1992\"],\n            MEMBER[\"European Terrestrial Reference Frame 1993\"],\n            MEMBER[\"European Terrestr

### Creating a STAC collection

Now, we will construct and register the STAC collection from scratch using the `pystac` library. The STAC collection is used to store all of the assets of our dataset.

In [6]:
from datetime import datetime
import pystac
 

spatial_extent = pystac.SpatialExtent(bboxes=[[-90, -180, 90, 180]])
temporal_extent = pystac.TemporalExtent(intervals=[[datetime(2020,1,1), datetime(2020,1,1)]])

description_markdown = """
Collection containing Habitat maps as input for Ecosystem Extent Accounts for a number of demonstrator countries. 
The Habitat Mapping within these datasets is based on the EUNIS habitat types hierarchical view of 2012 (https://eunis.eea.europa.eu/habitats.jsp)
"""

collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)
collection = pystac.Collection(
    id='esa-people-ea-habitat-maps',
    title="Ecosystem Habitat Maps (EUNIS 2012)",
    description=description_markdown,
    extent=pystac.Extent(spatial_extent, temporal_extent),
    license="CC-BY-4.0",
)

print(json.dumps(collection.to_dict(), indent=2))

{
  "type": "Collection",
  "id": "esa-people-ea-habitat-maps",
  "stac_version": "1.1.0",
  "description": "\nCollection containing Habitat maps as input for Ecosystem Extent Accounts for a number of demonstrator countries. \nThe Habitat Mapping within these datasets is based on the EUNIS habitat types hierarchical view of 2012 (https://eunis.eea.europa.eu/habitats.jsp)\n",
  "links": [],
  "title": "Ecosystem Habitat Maps (EUNIS 2012)",
  "extent": {
    "spatial": {
      "bbox": [
        [
          -90,
          -180,
          90,
          180
        ]
      ]
    },
    "temporal": {
      "interval": [
        [
          "2020-01-01T00:00:00Z",
          "2020-01-01T00:00:00Z"
        ]
      ]
    }
  },
  "license": "CC-BY-4.0"
}


To ensure the proper permissions are applied to our collection, we restrict write access to only the STAC administrators defined within our project. These administrators are assigned the `stac-admin-<environment>` role, granting them exclusive rights to add content to the collection.

Read permissions are set to public, allowing unrestricted access for viewing the collection.


In [7]:
coll_dict = collection.to_dict()

default_auth = {
    "_auth": {
        "read": ["anonymous"],
        "write": ["stac-admin-prod"]
    }
}

coll_dict.update(default_auth)

Finally, we send a request to the `/collections` endpoint to create the new collection

In [8]:
response = requests.post(f"{CATALOGUE}/collections", auth=auth,json=coll_dict)
response

<Response [201]>

We can verify the creation of the collection by querying the collections from the catalogue.

In [9]:
print(json.dumps(r.collections()['collections'][0], indent=2))


{
  "id": "esa-people-ea-habitat-maps",
  "description": "\nCollection containing Habitat maps as input for Ecosystem Extent Accounts for a number of demonstrator countries. \nThe Habitat Mapping within these datasets is based on the EUNIS habitat types hierarchical view of 2012 (https://eunis.eea.europa.eu/habitats.jsp)\n",
  "stac_version": "1.1.0",
  "links": [
    {
      "rel": "items",
      "type": "application/geo+json",
      "href": "https://catalogue.demo.apex.esa.int/collections/esa-people-ea-habitat-maps/items"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://catalogue.demo.apex.esa.int/"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://catalogue.demo.apex.esa.int/"
    },
    {
      "rel": "self",
      "type": "application/json",
      "href": "https://catalogue.demo.apex.esa.int/collections"
    }
  ],
  "title": "Ecosystem Habitat Maps (EUNIS 2012)",
  "type": "Collection",
  "licens

### Creating the STAC item

In this step, we will create and register a dedicated STAC item to represent an element from our dataset. As mentioned earlier, this approach is generally not recommended, as most tools automatically generate STAC metadata for you.

In [10]:
collection_item = create_stac_item(
    source="https://eoresults.esa.int/d/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012/2020/01/01/ESA_PEOPLE_EA_HABITAT_MAPS_EUNIS_2012-SK_2020/SK_L1_pp_2020.tif",
    collection=collection.id,
    id="esa-people-ea-habitatmap-sk",
    asset_name="result",
    with_proj=True,
    with_raster=True,
)

print(json.dumps(collection_item.to_dict(), indent=2))

{
  "type": "Feature",
  "stac_version": "1.1.0",
  "stac_extensions": [
    "https://stac-extensions.github.io/projection/v1.1.0/schema.json",
    "https://stac-extensions.github.io/raster/v1.1.0/schema.json"
  ],
  "id": "esa-people-ea-habitatmap-sk",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          16.14854432149387,
          47.76794093707247
        ],
        [
          23.624850820208717,
          47.08558308312211
        ],
        [
          24.322615943793686,
          49.5635338740363
        ],
        [
          16.471460358214575,
          50.2864943351419
        ],
        [
          16.14854432149387,
          47.76794093707247
        ]
      ]
    ]
  },
  "bbox": [
    16.14854432149387,
    47.08558308312211,
    24.322615943793686,
    50.2864943351419
  ],
  "properties": {
    "proj:epsg": 3035,
    "proj:geometry": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            4781560.0,
    

After creating the STAC metadata, we can now add it to our collection.

In [11]:
r.collection_item_create(collection.id, collection_item.to_dict())

True

We can verify the existing if the item was created by retrieving all of the collection items.

In [12]:
print(json.dumps(r.collection_items(collection.id), indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "stac_version": "1.1.0",
      "stac_extensions": [
        "https://stac-extensions.github.io/projection/v1.1.0/schema.json",
        "https://stac-extensions.github.io/raster/v1.1.0/schema.json"
      ],
      "id": "esa-people-ea-habitatmap-sk",
      "collection": "esa-people-ea-habitat-maps",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              16.14854432149387,
              47.76794093707247
            ],
            [
              23.624850820208717,
              47.08558308312211
            ],
            [
              24.322615943793686,
              49.5635338740363
            ],
            [
              16.471460358214575,
              50.2864943351419
            ],
            [
              16.14854432149387,
              47.76794093707247
            ]
          ]
        ]
      },
      "bbox": [
        16.148

## Visualizing in the APEx STAC Browser

The APEx product catalogue includes a [STAC browser](../instantiation/catalog.qmd#stac-browser) that enables you to visually explore collections and items. You can access the browser at the following URL: `browser.<project>.apex.esa.int`.

## Cleaning Up

Once the demo is complete, we can easily clean up by deleting the test collection, which is no longer needed.

In [13]:
requests.delete(f"{CATALOGUE}/collections/" + collection.id, auth=auth)

<Response [204]>